[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/tuanh118/uw-fsdl-bert-transfer-translation/blob/add-collab-notebook/Notebook.ipynb)

## Set up the Collab VM and import libraries.


In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

# Hack to reference other files in the repo in Collab.
!git clone https://github.com/tuanh118/uw-fsdl-bert-transfer-translation
import sys
sys.path.append('./uw-fsdl-bert-transfer-translation')

# Install required packages.
!pip install transformers

from BertTransformerModel import BertTransformerModel


Cloning into 'uw-fsdl-bert-transfer-translation'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 34 (delta 9), reused 23 (delta 5), pack-reused 0
Unpacking objects: 100% (34/34), done.
     |████████████████████████████████| 645kB 2.7MB/s 
     |████████████████████████████████| 3.8MB 11.5MB/s 
     |████████████████████████████████| 1.1MB 30.1MB/s 
     |████████████████████████████████| 890kB 35.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=f3455004a4f61e73d1d756c83a55250f00244f220d48052098f5f8f564e7e975
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


## Load, tokenize and split data into train/dev/test sets.


In [0]:
# TODO